In [1]:
import re
import pandas as pd

In [2]:
### กำหนด data type ที่เหมาะสมกับ attribute values (Custom data types) ###


def guess_column_types(file_path, delimiter=',', has_headers=True):
    try:
        # Read the CSV file using the specified delimiter and header settings
        df = pd.read_csv(file_path, sep=delimiter,low_memory=False, header=0 if has_headers else None)

        # Initialize a dictionary to store column data types
        column_types = {}

        # Loop through columns and infer data types
        for column in df.columns:
            # sample_values = df[column].dropna().sample(min(5, len(df[column])), random_state=42)

            # Check for datetime format "YYYY-MM-DD HH:MM:SS"
            is_datetime = all(re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', str(value)) for value in df[column])

            # Check for date format "YYYY-MM-DD"
            is_date = all(re.match(r'\d{4}-\d{2}-\d{2}', str(value)) for value in df[column])

            # Assign data type based on format detection
            if is_datetime:
                inferred_type = 'datetime64'
            elif is_date:
                inferred_type = 'date'
            else:
                inferred_type = pd.api.types.infer_dtype(df[column], skipna=True)

            column_types[column] = inferred_type

        return (True, column_types)  # Return success and column types
    except pd.errors.ParserError:
        return (False, str(e))  # Return error message

In [3]:
file_path = '../data/LoanStats_web.csv'
result, column_types_or_error = guess_column_types(file_path)

if result:
    print("Column Types:", column_types_or_error)
else:
    print("Error:", column_types_or_error)


Column Types: {'id': 'string', 'member_id': 'floating', 'loan_amnt': 'floating', 'funded_amnt': 'floating', 'funded_amnt_inv': 'floating', 'term': 'string', 'int_rate': 'string', 'installment': 'floating', 'grade': 'string', 'sub_grade': 'string', 'emp_title': 'string', 'emp_length': 'string', 'home_ownership': 'string', 'annual_inc': 'floating', 'verification_status': 'string', 'issue_d': 'string', 'loan_status': 'string', 'pymnt_plan': 'string', 'url': 'floating', 'desc': 'string', 'purpose': 'string', 'title': 'string', 'zip_code': 'string', 'addr_state': 'string', 'dti': 'floating', 'delinq_2yrs': 'floating', 'earliest_cr_line': 'string', 'inq_last_6mths': 'floating', 'mths_since_last_delinq': 'floating', 'mths_since_last_record': 'floating', 'open_acc': 'floating', 'pub_rec': 'floating', 'revol_bal': 'floating', 'revol_util': 'string', 'total_acc': 'floating', 'initial_list_status': 'string', 'out_prncp': 'floating', 'out_prncp_inv': 'floating', 'total_pymnt': 'floating', 'total_p

In [4]:
# เปลี่ยน data type บางตัว ให้เหมาะสมกับ python's environment และ MSSQL
column_types_corrected = {col: ('datetime64' if t == 'date' else 'float64' if t == 'floating' else t) \
                          for col, t in column_types_or_error.items()}

In [5]:
### นำ dictionary ที่บรรจุ ชื่อ col และ data type มาใช้เป็น parameter ของการทำ pd.read_csv ( ) อ่าน csv มาเป็น dataframe ###
raw_df = pd.read_csv('../data/LoanStats_web.csv', dtype=column_types_corrected)

In [6]:
# คำนวน percentage ของ missing values ของแต่ละ col. ใน dataframe (raw_df)
missing_percentage = raw_df.isnull().mean() * 100

# กรอง columns ที่มี null เกินกว่า 30% ออกไป
columns_to_keep = missing_percentage[missing_percentage <= 30].index.tolist()
filteredCol_df = raw_df[columns_to_keep]

In [7]:
raw_df = filteredCol_df

In [8]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432466 entries, 0 to 1432465
Data columns (total 100 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   loan_amnt                   1432440 non-null  float64
 1   funded_amnt                 1432440 non-null  float64
 2   funded_amnt_inv             1432440 non-null  float64
 3   term                        1432440 non-null  string 
 4   int_rate                    1432440 non-null  string 
 5   installment                 1432440 non-null  float64
 6   grade                       1432440 non-null  string 
 7   sub_grade                   1432440 non-null  string 
 8   emp_title                   1302202 non-null  string 
 9   emp_length                  1323970 non-null  string 
 10  home_ownership              1432440 non-null  string 
 11  annual_inc                  1432440 non-null  float64
 12  verification_status         1432440 non-null  string 
 

In [12]:
raw_df.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc

In [14]:
# สร้างรายการของคอลัมน์ที่มี Non-Null จำนวน 1,432,440
## selected_columns = [col for col in raw_df.columns if raw_df[col].notnull().sum() == 1432440]
selected_columns = [col for col in raw_df.columns if raw_df[col].isnull().sum() <= 26]

# แสดงคอลัมน์ที่เลือก
print("Selected columns:", selected_columns)

# สร้าง DataFrame ใหม่จากคอลัมน์ที่เลือก
df_selected = raw_df[selected_columns]


# ลบแถวที่มีค่า null ในคอลัมน์เหล่านั้น
nullNull_df = df_selected.dropna()

# แสดงข้อมูลทั่วไปของ DataFrame หลังจากลบ null
nullNull_df.info()

Selected columns: ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'addr_state', 'delinq_2yrs', 'earliest_cr_line', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_30dpd', 

In [15]:
import pandas as pd

# สมมติว่า df ได้รับการกำหนดค่าอย่างถูกต้องตามโค้ดก่อนหน้านี้

# แปลงประเภทข้อมูลและสร้างสำเนาใหม่ทันทีเพื่อใช้งาน
df_prepared = nullNull_df.copy()
df_prepared['issue_d'] = pd.to_datetime(df_prepared['issue_d'], format='%b-%Y')

if df_prepared['int_rate'].dtype == 'string':
    df_prepared['int_rate'] = df_prepared['int_rate'].str.rstrip('%').astype('float') / 100.0

# สร้าง Dimension Tables
home_ownership_dim = df_prepared[['home_ownership']].drop_duplicates().reset_index(drop=True)
home_ownership_dim.reset_index(inplace=True)
home_ownership_dim['home_ownership_id'] = home_ownership_dim.index

loan_status_dim = df_prepared[['loan_status']].drop_duplicates().reset_index(drop=True)
loan_status_dim.reset_index(inplace=True)
loan_status_dim['loan_status_id'] = loan_status_dim.index

# สร้าง Dimension Table สำหรับ issue_d
issue_d_dim = df_prepared[['issue_d']].drop_duplicates().reset_index(drop=True)

issue_d_dim['month'] = issue_d_dim['issue_d'].dt.month
issue_d_dim['year'] = issue_d_dim['issue_d'].dt.year


issue_d_dim.reset_index(inplace=True)
issue_d_dim['issue_d_id'] = issue_d_dim.index

# สร้าง mapping สำหรับ Dimension Tables
home_ownership_map = home_ownership_dim.set_index('home_ownership')['home_ownership_id'].to_dict()
loan_status_map = loan_status_dim.set_index('loan_status')['loan_status_id'].to_dict()
issue_d_map = issue_d_dim.set_index('issue_d')['issue_d_id'].to_dict()

# สร้าง Fact Table ใหม่จาก DataFrame ที่เตรียมไว้
loans_fact = df_prepared.copy()
loans_fact['home_ownership_id'] = loans_fact['home_ownership'].map(home_ownership_map)
loans_fact['loan_status_id'] = loans_fact['loan_status'].map(loan_status_map)
loans_fact['issue_d_id'] = loans_fact['issue_d'].map(issue_d_map)

# เลือกคอลัมน์ที่จำเป็นสำหรับ Fact Table
loans_fact = loans_fact[['application_type','loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', 'home_ownership_id', 'loan_status_id', 'issue_d_id']]


In [16]:
nullNull_df.groupby('issue_d').size()

issue_d
Apr-2016     36432
Apr-2018     42928
Aug-2016     36280
Aug-2018     46079
Dec-2016     36183
Dec-2018     40134
Feb-2016     39529
Feb-2017    111052
Feb-2018     32746
Feb-2019     35985
Jan-2016     32366
Jan-2017    127340
Jan-2018     36347
Jan-2019     43584
Jul-2016     34696
Jul-2018     43089
Jun-2016     33019
Jun-2018     41533
Mar-2016     61992
Mar-2017    148724
Mar-2018     38771
Mar-2019     36106
May-2016     28403
May-2018     46311
Nov-2016     34591
Nov-2018     41973
Oct-2016     32772
Oct-2018     46305
Sep-2016     28144
Sep-2018     39026
dtype: int64

In [17]:
df_prepared.groupby('loan_status').size()

loan_status
Charged Off           147348
Current               702223
Default                   27
Fully Paid            551955
In Grace Period         7928
Late (16-30 days)       4207
Late (31-120 days)     18752
dtype: int64

In [18]:
df_prepared.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432440 entries, 0 to 1432463
Data columns (total 68 columns):
 #   Column                      Non-Null Count    Dtype         
---  ------                      --------------    -----         
 0   loan_amnt                   1432440 non-null  float64       
 1   funded_amnt                 1432440 non-null  float64       
 2   funded_amnt_inv             1432440 non-null  float64       
 3   term                        1432440 non-null  string        
 4   int_rate                    1432440 non-null  float64       
 5   installment                 1432440 non-null  float64       
 6   grade                       1432440 non-null  string        
 7   sub_grade                   1432440 non-null  string        
 8   home_ownership              1432440 non-null  string        
 9   annual_inc                  1432440 non-null  float64       
 10  verification_status         1432440 non-null  string        
 11  issue_d                     1

In [19]:
df_prepared.groupby('int_rate').size()

int_rate
0.0531     8613
0.0532    36953
0.0600      493
0.0607     5019
0.0608     2968
          ...  
0.3079     1223
0.3084      676
0.3089      484
0.3094      324
0.3099      464
Length: 258, dtype: int64

In [20]:
import pandas as pd

# กำหนดข้อมูลและเตรียมข้อมูล df_prepared, loans_fact, home_ownership_dim, loan_status_dim, และ issue_d_dim

# Join `loans_fact` กับ `home_ownership_dim` โดยใช้ 'home_ownership_id'
loans_fact_with_home_ownership = pd.merge(loans_fact, home_ownership_dim, on='home_ownership_id', how='left', suffixes=('', '_home_ownership'))

# Join ผลลัพธ์กับ `loan_status_dim` โดยใช้ 'loan_status_id'
loans_fact_with_home_ownership_with_loan_status = pd.merge(loans_fact_with_home_ownership, loan_status_dim, on='loan_status_id', how='left', suffixes=('', '_loan_status'))

# Join ผลลัพธ์กับ `issue_d_dim` โดยใช้ 'issue_d_id'
final_df = pd.merge(loans_fact_with_home_ownership_with_loan_status, issue_d_dim, on='issue_d_id', how='left', suffixes=('', '_issue_d'))


In [21]:
print(f"จำนวนแถวใน nullNull_df: {nullNull_df.shape[0]}")
print(f"จำนวนแถวใน final_df: {final_df.shape[0]}")


จำนวนแถวใน nullNull_df: 1432440
จำนวนแถวใน final_df: 1432440


In [22]:
print("การเปรียบเทียบค่า loan_amnt แบบสุ่ม:")
print(nullNull_df['loan_amnt'].head())
print(final_df['loan_amnt'].head())


การเปรียบเทียบค่า loan_amnt แบบสุ่ม:
0    12800.0
1     9000.0
2     8000.0
3    25000.0
4    35000.0
Name: loan_amnt, dtype: float64
0    12800.0
1     9000.0
2     8000.0
3    25000.0
4    35000.0
Name: loan_amnt, dtype: float64


In [23]:
print("จำนวนค่า Null ใน final_df หลังจากการ join:")
print(final_df.isnull().sum())


จำนวนค่า Null ใน final_df หลังจากการ join:
application_type     0
loan_amnt            0
funded_amnt          0
term                 0
int_rate             0
installment          0
home_ownership_id    0
loan_status_id       0
issue_d_id           0
index                0
home_ownership       0
index_loan_status    0
loan_status          0
index_issue_d        0
issue_d              0
month                0
year                 0
dtype: int64


In [24]:
loans_fact

,application_type,loan_amnt,funded_amnt,term,int_rate,installment,home_ownership_id,loan_status_id,issue_d_id
0,Individual,12800.0,12800.0,36 months,0.1199,425.09,0,0,0
1,Individual,9000.0,9000.0,36 months,0.1531,313.36,1,0,0
2,Individual,8000.0,8000.0,36 months,0.1075,260.97,1,0,0
3,Individual,25000.0,25000.0,60 months,0.1299,568.70,1,1,0
4,Individual,35000.0,35000.0,60 months,0.1825,893.54,0,0,0
...,...,...,...,...,...,...,...,...,...
1432459,Individual,5000.0,5000.0,36 months,0.1356,169.83,0,2,29
1432460,Individual,6000.0,6000.0,36 months,0.0646,183.79,0,2,29
1432461,Individual,16000.0,16000.0,36 months,0.1614,563.62,0,2,29
1432462,Individual,16000.0,16000.0,60 months,0.1131,350.36,0,2,29


In [25]:
loans_fact.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432440 entries, 0 to 1432463
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   application_type   1432440 non-null  string 
 1   loan_amnt          1432440 non-null  float64
 2   funded_amnt        1432440 non-null  float64
 3   term               1432440 non-null  string 
 4   int_rate           1432440 non-null  float64
 5   installment        1432440 non-null  float64
 6   home_ownership_id  1432440 non-null  int64  
 7   loan_status_id     1432440 non-null  int64  
 8   issue_d_id         1432440 non-null  int64  
dtypes: float64(4), int64(3), string(2)
memory usage: 109.3 MB


In [26]:
loan_status_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           7 non-null      int64 
 1   loan_status     7 non-null      string
 2   loan_status_id  7 non-null      int64 
dtypes: int64(2), string(1)
memory usage: 296.0 bytes


In [27]:
issue_d_dim.drop(columns=['index'], inplace=True)

issue_d_dim

,issue_d,month,year,issue_d_id
0,2016-03-01,3,2016,0
1,2016-02-01,2,2016,1
2,2016-01-01,1,2016,2
3,2016-06-01,6,2016,3
4,2016-05-01,5,2016,4
5,2016-04-01,4,2016,5
6,2016-09-01,9,2016,6
7,2016-08-01,8,2016,7
8,2016-07-01,7,2016,8
9,2016-12-01,12,2016,9


In [28]:
loan_status_dim.drop(columns=['index'], inplace=True)

loan_status_dim

,loan_status,loan_status_id
0,Fully Paid,0
1,Charged Off,1
2,Current,2
3,Late (31-120 days),3
4,Default,4
5,Late (16-30 days),5
6,In Grace Period,6


In [29]:
home_ownership_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              5 non-null      int64 
 1   home_ownership     5 non-null      string
 2   home_ownership_id  5 non-null      int64 
dtypes: int64(2), string(1)
memory usage: 248.0 bytes


In [30]:
home_ownership_dim.drop(columns=['index'], inplace=True)

home_ownership_dim

,home_ownership,home_ownership_id
0,MORTGAGE,0
1,RENT,1
2,OWN,2
3,ANY,3
4,NONE,4


In [31]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432440 entries, 0 to 1432439
Data columns (total 17 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   application_type   1432440 non-null  string        
 1   loan_amnt          1432440 non-null  float64       
 2   funded_amnt        1432440 non-null  float64       
 3   term               1432440 non-null  string        
 4   int_rate           1432440 non-null  float64       
 5   installment        1432440 non-null  float64       
 6   home_ownership_id  1432440 non-null  int64         
 7   loan_status_id     1432440 non-null  int64         
 8   issue_d_id         1432440 non-null  int64         
 9   index              1432440 non-null  int64         
 10  home_ownership     1432440 non-null  string        
 11  index_loan_status  1432440 non-null  int64         
 12  loan_status        1432440 non-null  string        
 13  index_issue_d      1432440 

In [32]:
final_df

,application_type,loan_amnt,funded_amnt,term,int_rate,installment,home_ownership_id,loan_status_id,issue_d_id,index,home_ownership,index_loan_status,loan_status,index_issue_d,issue_d,month,year
0,Individual,12800.0,12800.0,36 months,0.1199,425.09,0,0,0,0,MORTGAGE,0,Fully Paid,0,2016-03-01,3,2016
1,Individual,9000.0,9000.0,36 months,0.1531,313.36,1,0,0,1,RENT,0,Fully Paid,0,2016-03-01,3,2016
2,Individual,8000.0,8000.0,36 months,0.1075,260.97,1,0,0,1,RENT,0,Fully Paid,0,2016-03-01,3,2016
3,Individual,25000.0,25000.0,60 months,0.1299,568.70,1,1,0,1,RENT,1,Charged Off,0,2016-03-01,3,2016
4,Individual,35000.0,35000.0,60 months,0.1825,893.54,0,0,0,0,MORTGAGE,0,Fully Paid,0,2016-03-01,3,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432435,Individual,5000.0,5000.0,36 months,0.1356,169.83,0,2,29,0,MORTGAGE,2,Current,29,2019-01-01,1,2019
1432436,Individual,6000.0,6000.0,36 months,0.0646,183.79,0,2,29,0,MORTGAGE,2,Current,29,2019-01-01,1,2019
1432437,Individual,16000.0,16000.0,36 months,0.1614,563.62,0,2,29,0,MORTGAGE,2,Current,29,2019-01-01,1,2019
1432438,Individual,16000.0,16000.0,60 months,0.1131,350.36,0,2,29,0,MORTGAGE,2,Current,29,2019-01-01,1,2019


In [33]:
server = '34.136.225.236'
database = 'loanDW'
username = 'SA'
password = 'Passw0rd123456'